In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import requests.exceptions
from urllib.parse import urlsplit
from urllib.parse import urlparse
from collections import deque


In [7]:
def extract_contact_info(html):
 
 email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
 phone_pattern = re.compile(r'\+\d+[-\.\s]??\d+[-\.\s]??\d+[-\.\s]??\d+[-\.\s]??\d+[-\.\s]??\d+')
 emails = set(re.findall(email_pattern, soup.text))
 phones = set(re.findall(phone_pattern, soup.text))
 return emails, phones


def get_contact_info(url):
 response = requests.get(url)
 if response.status_code == 200:
        contacts = extract_contact_info(response.text)
        emails = ', '.join(contacts[0]) if contacts[0] else 'N/A'
        phones = ', '.join(contacts[1]) if contacts[1] else 'N/A'
        # Check if emails and phones lists are empty
        if not emails:
            emails = 'N/A'
        if not phones:
            phones = 'N/A'
 return emails, phones



In [8]:
# MAIN

url = "https://orange.tn" #put the targeted company's url here
# a queue of urls to be crawled
new_urls = deque([url])

# a set of urls that we have already been processed 
processed_urls = set()
# a set of domains inside the target website
local_urls = set()
# a set of domains outside the target website
foreign_urls = set()
# a set of broken urls
broken_urls = set()
# process urls one by one until we exhaust the queue
while len(new_urls):
    # move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    processed_urls.add(url)
    # get url's content
    print("Processing %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidURL, requests.exceptions.InvalidSchema):
        # add broken urls to it's own set, then continue
        broken_urls.add(url)
        continue
    
    # extract base url to resolve relative links
    parts = urlsplit(url)
    base = "{0.netloc}".format(parts)
    strip_base = base.replace("www.", "")
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, "lxml")
    for link in soup.find_all('a'):
        # extract link url from the anchor
        anchor = link.attrs["href"] if "href" in link.attrs else ''
        if anchor.startswith('/contact'):
            local_link = base_url + anchor
            local_urls.add(local_link)
        for i in local_urls:
            if not i in new_urls and not i in processed_urls:
                new_urls.append(i)


Processing https://orange.tn
Processing https://orange.tn/contact


In [9]:
for item in processed_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    html = response.text
    contact_info = get_contact_info(item)
    if contact_info is not None:
        emails, phones = contact_info
        print('Emails:', emails)
        print('Phones:', phones)
        break

Emails: N/A
Phones: +21631111150, +21631111151, +216 3001 3001
